<a href="https://colab.research.google.com/github/mauriciomm7/datajourney/blob/main/ECPR21_MS03_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Class Project 
### **ECPR Summer 2021**
MS03: Python Programming for Social Scientists

Instructures: *Orsolya Vasarhelyi* and *Rebeka O. Szabo* 

Mauricio Mandujano Manriquez

### Outline 
1.   Project Description
2.   Creating the Web scrapper
2.   Describing the data: *PS: Political Science & Politics*
3.   Preparing the data: Genders and number of authors.
4.   Preliminary correlatiation analysis
---


**NOTE:** To render plots and charts please open directly in Google Collab.

## 1 Project Description 
This Jupyter Notebook contains my final class project for the Summer course *MS03: Python Programming for Social Scientists* by the ECPR in association with KU Leuven. The main objective of this project is to consolidate my newfound skillset of data, collection, processing and manipulation from different sources such as websites or APIs.

Thus, for this project I will be working with scrapped web data from the Cambridge Core series journals on Political Science. The aim of this project is to describe patterns found across the *most cited* journals.


In this work **Most Cited** referes to self defined category of journals as listed on the Cambridge Core website. According to [Cambridge University Press](https://www.cambridge.org/core/journals/american-political-science-review/most-cited?pageNum=278&searchWithinIds=833A7242AC7B607BA7F6168DA072DB3B%2CDEF6D9E6432F4E804AD2D9416C8C2165&productType=JOURNAL_ARTICLE&pageSize=20&filters%5BisCitedByMin%5D=0&template=cambridge-core%2Fjournal%2Farticle-listings%2Flistings-wrapper&displayNasaAds=false&showCitationNumbers=true&suppressArticleTypeGrouping=true&sort=platformMetadata.citationCount.crossRef%3Adesc). This page lists the all time most cited articles for any given journal. 

### Objectives
**Create a database of the the most cited journals from a Cambridge Core Journal.**

Target attributes: 
- Title 
- Authors 
- Publication Data 
- Citations

**Create adidtional attributes from scrapped data**

Additional attributes: 
- Number of authors
- Gender of author(s)
- Publication year

**Create Preliminary Data Analysis and Visualization** 
- Descriptive Statistics
- Data Visualizations

## 2 Creating the Web Scrapper 


In [ ]:
# This step is optional but gives you the advantage of saving all output files directly on drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd 'drive/My Drive/ECPR_SummerSchool2021/ECPR_summer2021_mandujano/PROJECT/'

/content/drive/My Drive/ECPR_SummerSchool2021/ECPR_summer2021_mandujano/PROJECT


In [ ]:
!# Required libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [ ]:
#Test links, 
apsr ='https://www.cambridge.org/core/journals/american-political-science-review/most-cited?pageNum=1&searchWithinIds=833A7242AC7B607BA7F6168DA072DB3B%2CDEF6D9E6432F4E804AD2D9416C8C2165&productType=JOURNAL_ARTICLE&pageSize=20&filters%5BisCitedByMin%5D=0&template=cambridge-core%2Fjournal%2Farticle-listings%2Flistings-wrapper&displayNasaAds=false&showCitationNumbers=true&suppressArticleTypeGrouping=true&sort=platformMetadata.citationCount.crossRef%3Adesc'
ps= 'https://www.cambridge.org/core/journals/ps-political-science-and-politics/most-cited?pageNum=1&searchWithinIds=DACDE76A4E1D287359185FE191ACDEDC%2C0C26D20831BAFEE406A1D823C28634B7%2CC8CE5A2136AF3E5DBFC03D4EABA2F773&productType=JOURNAL_ARTICLE&pageSize=20&filters%5BisCitedByMin%5D=0&template=cambridge-core%2Fjournal%2Farticle-listings%2Flistings-wrapper&displayNasaAds=false&showCitationNumbers=true&suppressArticleTypeGrouping=true&sort=platformMetadata.citationCount.crossRef%3Adesc'
pa ='https://www.cambridge.org/core/journals/political-analysis/most-cited?pageNum=1&searchWithinIds=EEF1D4438BAB8498B2A647F9DBC47AA7&productType=JOURNAL_ARTICLE&pageSize=20&filters%5BisCitedByMin%5D=0&template=cambridge-core%2Fjournal%2Farticle-listings%2Flistings-wrapper&displayNasaAds=false&showCitationNumbers=true&suppressArticleTypeGrouping=true&sort=platformMetadata.citationCount.crossRef%3Adesc'

In [ ]:
def cambrige_coredatabase (url, last):
  """  
  **url:** Link of the first page of the Cambrige Core journal most cited page.
  dtype: str 

  **last:**  number of last page to parse
  dtype: int
  """
  globaldf=[]
  cambridgecore_link = str(url)
  splitted=cambridgecore_link.split('=1')

  links_toparse= []
  for page in range(0, last):
    new_link=splitted[0]+'='+str(page)+splitted[1]
    links_toparse.append(new_link)
  def format_to_YYYYMMDD (date):
    """
    date =  'DD month YYYY' only.
    Example: 18 October 2011
    """
    split = date.split(" ")
    months = {'January':'01' , 'February':'02', 'March':'03', 'April':'04', 'May':'05', 'June':'06', 'July':'07', 'August':'08', 'September':'09', 'October':'10','November':'11','December':'12'}
    date2=date.split(" ")[2]
    num = months.get(date2)
    split[2]= num
    output=str('-'.join(split))
    return (output[1:])
  
  for link in links_toparse:
    text = requests.get(cambridgecore_link).text
    soup = BeautifulSoup(text, 'lxml')
    clean_authors= []
    authors = soup.find_all('li', {'class':"author"})
    for author in authors:
      cauthor=str(author.text).replace("\n", "")
      clean_authors.append(cauthor)

    #find all titles, clean them and make put them into a list 
    clean_titles =[]
    titles = soup.find_all('h5')
    for title in titles:
      ctitle=str(title.text).replace("\n", "")# this is necessary to get rid of unwanted /n characters 
      clean_titles.append(ctitle) 

    #Citations
    clean_citations =[]
    citations = soup.find_all('span',{'class':"number"})
    for cite in citations:
      cite = int(cite.text)
      clean_citations.append(cite)

    # publication dates
    clean_dates=[]
    dates = soup.find_all('span',{'class':"date"})
    for date in dates:
      cdate = str(date.text)
      cdate = format_to_YYYYMMDD(cdate) 
      clean_dates.append(cdate)
    clean_dates= pd.to_datetime(clean_dates, dayfirst=True)
    data=dict={"Titles": clean_titles, "Authors":clean_authors,"Citations":clean_citations, 'Publication Date':clean_dates }
    df=pd.DataFrame(data) 
    globaldf.append(df)
  output = pd.concat(globaldf)
  output = output.reset_index(drop=True)
  return (output)

In [ ]:
ps_df = cambrige_coredatabase(ps, 102)
ps_df

,Titles,Authors,Citations,Publication Date
0,Understanding Process Tracing,David Collier,419,2011-10-18
1,Process Tracing and Elite Interviewing: A Case...,Oisín Tansey,194,2007-10-03
2,Conducting and Coding Elite Interviews,"Joel D. Aberbach, Bert A. Rockman",160,2003-01-02
3,Asking Questions: Techniques for Semistructure...,Beth L. Leech,155,2003-01-02
4,"Habits from Home, Lessons from School: Influen...","Molly W. Andolina, Krista Jenkins, Cliff Zukin...",136,2003-04-15
...,...,...,...,...
2035,The Influence of Family Political Discussion o...,"Hugh McIntosh, Daniel Hart, James Youniss",77,2007-07-10
2036,Controversies about Controversial Issues in De...,Diana E. Hess,75,2004-04-19
2037,Preference Gaps and Inequality in Representation,Martin Gilens,72,2009-04-02
2038,Explaining the Trump Vote: The Effect of Racis...,"Marc Hooghe, Ruth Dassonneville",67,2018-04-12


In [ ]:
apsr_df = cambrige_coredatabase(apsr, 278)
apsr_df

,Titles,Authors,Citations,Publication Date
0,What To Do (and Not to Do) with Time-Series Cr...,"Nathaniel Beck, Jonathan N. Katz",3185,2012-09-25
1,"Ethnicity, Insurgency, and Civil War","JAMES D. FEARON, DAVID D. LAITIN",3027,2003-03-12
2,"Increasing Returns, Path Dependence, and the S...",Paul Pierson,2775,2014-08-01
3,Some Social Requisites of Democracy: Economic ...,Seymour Martin Lipset,2482,2012-09-25
4,Two Faces of Power1,"Peter Bachrach, Morton S. Baratz",1665,2012-09-25
...,...,...,...,...
5555,Bargaining in Legislatures,"David P. Baron, John A. Ferejohn",961,2014-08-01
5556,Constituency Influence in Congress*,"Warren E. Miller, Donald E. Stokes",954,2014-08-01
5557,A Theory of the Calculus of Voting*,"William H. Riker, Peter C. Ordeshook",948,2017-04-08
5558,The Expansion of the Public Economy: A Compara...,David R. Cameron,928,2014-08-01


In [ ]:
pa_df= cambrige_coredatabase(pa, 36)
pa_df 

,Titles,Authors,Citations,Publication Date
0,Understanding Interaction Models: Improving Em...,"Thomas Brambor, William Roberts Clark, Matt Go...",3106,2017-01-04
1,Evaluating Online Labor Markets for Experiment...,"Adam J. Berinsky, Gregory A. Huber, Gabriel S....",2037,2017-01-04
2,Matching as Nonparametric Preprocessing for Re...,"Daniel E. Ho, Kosuke Imai, Gary King, Elizabet...",1918,2017-01-04
3,Logistic Regression in Rare Events Data,"Gary King, Langche Zeng",1860,2017-01-04
4,Causal Inference without Balance Checking: Coa...,"Stefano M. Iacus, Gary King, Giuseppe Porro",1173,2017-01-04
...,...,...,...,...
715,Estimating Latent Structure Models with Catego...,"Annabel Bolck, Marcel Croon, Jacques Hagenaars",356,2017-01-04
716,A Tale of Two Cultures: Contrasting Quantitati...,"James Mahoney, Gary Goertz",310,2017-01-04
717,Estimating Regression Models in Which the Depe...,"Jeffrey B. Lewis, Drew A. Linzer",308,2017-01-04
718,Which Elections Can Be Lost?,"Susan D. Hyde, Nikolay Marinov",287,2017-01-04


## 3 Preparing the Data

Once webscrapper has collected the journals from the **Most Cited** section on the Cambrige Core website and saved it as csv. In this section I create a fuction that processes the date for prelimiarly data analysis. Particularly, I am interested in accomplisng the following: 
1. Having the desired values in the dataframe.
2. Casting into the desired dtypes.
3. Creating any necesasary columns for data analysis.


In [ ]:
# Convert dataframes to csvs and save them in Google Drive
ps_df.to_csv('psdata.csv')
apsr_df.to_csv('apsrdata.csv')
pa_df.to_csv('padata.csv')
! cp psdata.csv "drive/My Drive/ECPR_SummerSchool2021/ECPR_summer2021_mandujano/PROJECT"
! cp apsrdata.csv "drive/My Drive/ECPR_SummerSchool2021/ECPR_summer2021_mandujano/PROJECT"
! cp padata.csv "drive/My Drive/ECPR_SummerSchool2021/ECPR_summer2021_mandujano/PROJECT"

NameError: ignored

In [ ]:
#Intall the gender guesser packege in the Collab environment
!pip install gender-guesser

In [ ]:
# Import necessary libraries
import pandas as pd
from datetime import datetime
import gender_guesser.detector as gender
ge = gender.Detector(case_sensitive=False)
from collections import Counter

In [ ]:
#fuction that processes and preapares data 
def prepare_data(journals_csv):
  mostcited = pd.read_csv(journals_csv) #read the file 
  mostcited = mostcited.rename(columns={'Unnamed: 0' : 'ID'})
  #Adding column for the number of authors 
  num_authors=[]
  for author in mostcited['Authors']:
    temp= len(author.split(", "))
    num_authors.append(temp)
  mostcited['num_authors'] = num_authors
  #Adding column with genders 
  genders = []
  for author in mostcited['Authors']:
    temp= author.split(", ")
    tempo=[]
    for person in temp:
      splitting= person.split(" ")
      firstname=splitting[0]
      tempo.append(ge.get_gender(firstname))
    genders.append(tempo)
  mostcited['genders']= genders
  mostcited = mostcited.rename (columns={'Publication Date' : 'publication_date'})#name was bugging me
  #Create a Column with only years
  pub_year=[]
  for date in mostcited['publication_date']:
    casted= int(date[:4])
    pub_year.append(casted)
  mostcited['publication_year'] = pub_year
  #create a fuction to that extracts the gender of publications where each number corresponds to a gender 1 = 'all male publication', 2 = 'all female publicatio', 3 = 'mixed publication', 4 = 'unkown' 
  temporal=[]
  for item in mostcited['genders']:
    if len(item) == 1: 
      if item[0]== 'male':
        temporal.append('Male')
      elif item[0] == 'female':
        temporal.append('Female')
      else:
        temporal.append('Unkown')
    elif 'male' in item and 'female' in item:
      temporal.append('Both')
    elif 'male' in item and 'female' not in item:
      temporal.append('Male')
    elif 'female' in item and 'male' not in item:
      temporal.append('Female')
    else:
      temporal.append('Unkown')
  mostcited['gender'] = temporal
  return (mostcited)

In [ ]:
psdata_ready= prepare_data('psdata.csv')
psdata_ready.head(10)

,ID,Titles,Authors,Citations,publication_date,num_authors,genders,publication_year,gender
0,0,Understanding Process Tracing,David Collier,419,2011-10-18,1,[male],2011,Male
1,1,Process Tracing and Elite Interviewing: A Case...,Oisín Tansey,194,2007-10-03,1,[male],2007,Male
2,2,Conducting and Coding Elite Interviews,"Joel D. Aberbach, Bert A. Rockman",160,2003-01-02,2,"[male, male]",2003,Male
3,3,Asking Questions: Techniques for Semistructure...,Beth L. Leech,155,2003-01-02,1,[female],2003,Female
4,4,"Habits from Home, Lessons from School: Influen...","Molly W. Andolina, Krista Jenkins, Cliff Zukin...",136,2003-04-15,4,"[female, female, male, male]",2003,Both
5,5,Validity and Reliability Issues In Elite Inter...,Jeffrey M. Berry,133,2003-01-02,1,[male],2003,Male
6,6,How Solid is Mass Support for Democracy—And Ho...,Ronald Inglehart,113,2017-04-25,1,[male],2017,Male
7,7,"The UnCivic Culture: Communication, Identity, ...",W. Lance Bennett,109,2013-09-02,1,[unknown],2013,Unkown
8,8,Anatomy of a Rally Effect: George W. Bush and ...,"Marc J. Hetherington, Michael Nelson",109,2017-04-25,2,"[male, male]",2017,Male
9,9,Expansive Citizenship—Voting beyond Territory ...,Rainer Bauböck,102,2005-10-06,1,[male],2005,Male


In [ ]:
apsrdata_ready= prepare_data('apsrdata.csv')
apsrdata_ready.head(10)

,ID,Titles,Authors,Citations,publication_date,num_authors,genders,publication_year,gender
0,0,What To Do (and Not to Do) with Time-Series Cr...,"Nathaniel Beck, Jonathan N. Katz",3185,2012-09-25,2,"[male, male]",2012,Male
1,1,"Ethnicity, Insurgency, and Civil War","JAMES D. FEARON, DAVID D. LAITIN",3027,2003-03-12,2,"[male, male]",2003,Male
2,2,"Increasing Returns, Path Dependence, and the S...",Paul Pierson,2775,2014-08-01,1,[male],2014,Male
3,3,Some Social Requisites of Democracy: Economic ...,Seymour Martin Lipset,2482,2012-09-25,1,[male],2012,Male
4,4,Two Faces of Power1,"Peter Bachrach, Morton S. Baratz",1665,2012-09-25,2,"[male, male]",2012,Male
5,5,"Dictatorship, Democracy, and Development",Mancur Olson,1448,2013-09-02,1,[unknown],2013,Unkown
6,6,A Behavioral Approach to the Rational Choice T...,Elinor Ostrom,1408,2014-08-01,1,[female],2014,Female
7,7,The New Institutionalism: Organizational Facto...,"James G. March, Johan P. Olsen",1309,2014-08-01,2,"[male, male]",2014,Male
8,8,Beyond SES: A Resource Model of Political Part...,"Henry E. Brady, Sidney Verba, Kay Lehman Schlo...",1192,2013-09-02,3,"[male, mostly_male, male]",2013,Male
9,9,A Method for Evaluating the Distribution of P...,"L. S. Shapley, Martin Shubik",1160,2012-09-25,2,"[unknown, male]",2012,Male


In [ ]:
padata_ready= prepare_data('padata.csv')
padata_ready.head(10) 

,ID,Titles,Authors,Citations,publication_date,num_authors,genders,publication_year,gender
0,0,Understanding Interaction Models: Improving Em...,"Thomas Brambor, William Roberts Clark, Matt Go...",3106,2017-01-04,3,"[male, male, male]",2017,Male
1,1,Evaluating Online Labor Markets for Experiment...,"Adam J. Berinsky, Gregory A. Huber, Gabriel S....",2037,2017-01-04,3,"[male, male, male]",2017,Male
2,2,Matching as Nonparametric Preprocessing for Re...,"Daniel E. Ho, Kosuke Imai, Gary King, Elizabet...",1918,2017-01-04,4,"[male, male, male, female]",2017,Both
3,3,Logistic Regression in Rare Events Data,"Gary King, Langche Zeng",1860,2017-01-04,2,"[male, unknown]",2017,Male
4,4,Causal Inference without Balance Checking: Coa...,"Stefano M. Iacus, Gary King, Giuseppe Porro",1173,2017-01-04,3,"[male, male, male]",2017,Male
5,5,Text as Data: The Promise and Pitfalls of Auto...,"Justin Grimmer, Brandon M. Stewart",920,2017-01-04,2,"[male, male]",2017,Male
6,6,Entropy Balancing for Causal Effects: A Multiv...,Jens Hainmueller,883,2017-01-04,1,[male],2017,Male
7,7,Latent Class Modeling with Covariates: Two Imp...,Jeroen K. Vermunt,813,2017-01-04,1,[male],2017,Male
8,8,Back to the Future: Modeling Time Dependence i...,"David B. Carter, Curtis S. Signorino",811,2017-01-04,2,"[male, male]",2017,Male
9,9,Set Relations in Social Research: Evaluating T...,Charles C. Ragin,652,2017-01-04,1,[male],2017,Male


## 4 Preliminary Data Analysis

In [ ]:
import altair as alt
import numpy as np
import seaborn as sns
import datetime 

In [ ]:
psdata_ready.describe()

,ID,Citations,num_authors,publication_year
count,2040.000000,2040.000000,2040.000000,2040.0000
mean,1019.500000,121.800000,1.600000,2007.7000
std,589.041594,75.980194,0.860443,5.2176
min,0.000000,66.000000,1.000000,2003.0000
25%,509.750000,80.000000,1.000000,2003.0000
50%,1019.500000,100.500000,1.000000,2005.5000
75%,1529.250000,133.750000,2.000000,2011.5000
max,2039.000000,419.000000,4.000000,2018.0000


In [ ]:
#Average Citations per paper 
alt.Chart(psdata_ready).mark_bar(size=15).encode(
    alt.Y('gender:N', title='Gender'),
    alt.X('average(Citations)'), color=alt.Color('gender:N', legend=None, scale=alt.Scale(scheme='tableau20')),
).properties(title= 'Average Citations in Political Science:PS')

alt.Chart(...)

In [ ]:
#Distribution of Paper Citations by gender 
alt.Chart(psdata_ready).mark_bar().encode(
    alt.X("Citations:Q", bin=alt.Bin(anchor=0, base=10, step=20)),
    alt.Y('count()', title='Number of papers'),
    column='gender',
    color=alt.Color('gender:N', legend=None, scale=alt.Scale(scheme='tableau20'))).properties(title= '# Citation Distribution of Published Papers in Political Science:PS',
    width=200,height=200)

alt.Chart(...)

In [ ]:
#Average Citations per Paper in Political Science by Gender
alt.Chart(psdata_ready).mark_area(point=True).encode( 
  alt.X('publication_date:T',title='Publication Date', ),
  alt.Y('average(Citations)', title= 'Average Citations per Paper'),
  color=alt.Color('gender:N', title= 'Gender',scale=alt.Scale(scheme='tableau20')
)).properties(title= 'Average Citations of "Most Cited" in Political Science:PS').interactive()

alt.Chart(...)

In [ ]:
alt.Chart(psdata_ready).mark_bar(size=10).encode(
    alt.X('publication_date:T'),
    alt.Y('average(Citations)'),
    color=alt.Color('gender:N', title='Gender', scale=alt.Scale(scheme='tableau20'))
).interactive()


alt.Chart(...)